In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from typing import Callable, List, Tuple
from abc import ABC, abstractmethod

In [ ]:
class Optimizer:
    """
    Base class for optimizers.
    """

    def __init__(self, *args, **kwargs) -> None:
        pass

    @abstractmethod
    def reset(self, theta_dim: int) -> None:
        """
        Reset the optimizer state.
        Should be implemented by subclasses.
        """
        pass

    def step(
        self,
        X: np.ndarray,
        Y: np.ndarray,
        g_grad: Callable,
        g_grad_and_hessian: Callable,
    ) -> None:
        """
        Perform one optimization step.
        Should be implemented by subclasses.
        """
        pass

In [ ]:
class Experiment:
    """
    Experiment class to run optimization experiments with second order methods,
    on a given function g, with computable gradient and hessian.
    """

    def __init__(
        self,
        g: Callable,
        g_grad: Callable,
        g_grad_and_hessian: Callable,
        optimizer_list: List[Optimizer],
        e: float,
        dataset: List[Tuple[np.ndarray, np.ndarray]] = None,
        generate_dataseet: Callable = None,
        true_theta: np.ndarray = None,
        true_hessian_inv: np.ndarray = None,
        theta_dim: int = None,
    ):
        """
        Initialize the experiment
        """
        if dataset is None and generate_dataseet is None:
            raise ValueError("dataset or create_dataset should be set")
        if true_theta is None and theta_dim is None:
            raise ValueError("dim_theta is not set")

        self.true_theta = true_theta
        self.true_hessian_inv = true_hessian_inv
        self.g = g
        self.g_grad = g_grad
        self.g_grad_and_hessian = g_grad_and_hessian
        self.optimizer_list = optimizer_list
        self.dataset = dataset
        self.generate_dataset = generate_dataseet
        self.e = e
        self.true_theta = true_theta
        self.theta_dim = theta_dim if true_theta is None else true_theta.shape[0]
        self.true_hessian_inv = true_hessian_inv

    def generate_initial_theta(self):
        """
        Generate a random initial theta
        """
        if self.e is None:
            raise ValueError("e is not set for generating random theta")
        loc = (
            self.true_theta if self.true_theta is not None else np.zeros(self.theta_dim)
        )
        self.initial_theta = loc + self.e * np.random.randn(self.theta_dim)

    def log_estimation_error(self, theta_errors, hessian_inv_errors, optimizer):
        if theta_errors is not None:
            theta_errors[optimizer.name].append(
                np.dot(self.theta - self.true_theta, self.theta - self.true_theta)
            )
        if hessian_inv_errors is not None and optimizer.hessian_inv is not None:
            hessian_inv_errors[optimizer.name].append(
                np.linalg.norm(optimizer.hessian_inv - self.true_hessian_inv, ord="fro")
            )

    def run(self, plot: bool = False) -> Tuple[List[float], List[float]]:
        """
        Run the experiment for a given initial theta, a dataset and a list of optimizers
        """
        if self.initial_theta is None:
            raise ValueError("initial theta is not set")
        if self.dataset is None:
            raise ValueError("dataset is not set")

        # Initialize the directories for errors if true values are provided
        theta_errors = (
            {optimizer.name: [] for optimizer in self.optimizer_list}
            if self.true_theta is not None
            else None
        )
        hessian_inv_errors = (
            {optimizer.name: [] for optimizer in self.optimizer_list}
            if self.true_hessian_inv is not None
            else None
        )

        # Run the experiment for each optimizer
        for optimizer in tqdm(self.optimizer_list, desc="Optimizers", leave=False):
            self.theta = self.initial_theta.copy()
            optimizer.reset(self.theta_dim)
            # Log initial error
            self.log_estimation_error(theta_errors, hessian_inv_errors, optimizer)

            # Online pass on the dataset
            for X, Y in tqdm(self.dataset, desc="Data", leave=False):
                optimizer.step(X, Y, self.theta, self.g_grad, self.g_grad_and_hessian)
                self.log_estimation_error(theta_errors, hessian_inv_errors, optimizer)
            # Convert errors to numpy arrays
            if theta_errors is not None:
                theta_errors[optimizer.name] = np.array(theta_errors[optimizer.name])
            if hessian_inv_errors is not None:
                hessian_inv_errors[optimizer.name] = np.array(
                    hessian_inv_errors[optimizer.name]
                )
        if plot:
            self.plot_errors(theta_errors, hessian_inv_errors)

        return theta_errors, hessian_inv_errors

    def run_multiple(
        self, num_runs: int = 100, n: int = 10_000
    ) -> Tuple[List[float], List[float]]:
        """
        Run the experiment multiple times by generating a new dataset and initial theta each time
        """
        if self.true_theta is None or self.generate_dataset is None:
            raise ValueError("true_theta and/or create_dataset are not set")

        # length of error arrays is n + 1 for initial error
        self.theta_errors_avg = (
            {optimizer.name: np.zeros(n + 1) for optimizer in self.optimizer_list}
            if self.true_theta is not None
            else None
        )
        self.hessian_inv_errors_avg = (
            {optimizer.name: np.zeros(n + 1) for optimizer in self.optimizer_list}
            if self.true_hessian_inv is not None
            else None
        )

        for i in tqdm(range(num_runs), desc="Runs"):
            self.dataset = self.generate_dataset(n, self.true_theta)
            self.generate_initial_theta()
            theta_errors, hessian_inv_errors = self.run()
            if self.true_theta is not None:
                for name, errors in theta_errors.items():
                    self.theta_errors_avg[name] += errors
            if self.true_hessian_inv is not None:
                for name, errors in hessian_inv_errors.items():
                    self.hessian_inv_errors_avg[name] += errors
        if self.true_theta is not None:
            for name, errors in self.theta_errors_avg.items():
                errors /= num_runs
        if self.true_hessian_inv is not None:
            for name, errors in self.hessian_inv_errors_avg.items():
                errors /= num_runs
        self.plot_errors(self.theta_errors_avg, self.hessian_inv_errors_avg)
        return self.theta_errors_avg, self.hessian_inv_errors_avg

    def plot_errors(self, theta_errors, hessian_inv_errors):
        """
        Plot the errors of estimated theta and hessian inverse of all optimizers
        """
        if self.true_theta is not None:
            for name, errors in theta_errors.items():
                plt.plot(errors, label=name)
            plt.xlabel("n")
            plt.ylabel("theta estimation squared error")
            plt.title(f"e = {self.e}")
            plt.legend()
            plt.show()
        if self.true_hessian_inv is not None:
            for name, errors in hessian_inv_errors.items():
                plt.plot(errors, label=name)
            plt.xlabel("n")
            plt.ylabel("hessian inverse estimation error")
            plt.title(f"e = {self.e}")
            plt.legend()
            plt.show()

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def create_dataset_logistic(n: int, true_theta: np.ndarray):
    d = len(true_theta)
    X = np.random.randn(n, d - 1)
    phi = np.hstack([np.ones((n, 1)), X])
    Y = np.random.binomial(1, sigmoid(phi @ true_theta))
    return list(zip(X, Y))  # Maybe use a dataloader for large datasets


def g(X: np.ndarray, Y: np.ndarray, h: np.ndarray):
    """
    Compute the logistic loss, works only for a batch of data
    """
    n, d = X.shape
    phi = np.hstack([np.ones(n, 1), X])
    dot_product = np.dot(phi, h)
    return np.log(1 + np.exp(dot_product)) - dot_product * Y


def g_grad(X: np.ndarray, Y: np.ndarray, h: np.ndarray):
    """
    Compute the gradient of the logistic loss, works only for a single data point
    """
    phi = np.hstack([np.ones((1,)), X])
    dot_product = np.dot(phi, h)
    p = sigmoid(dot_product)
    # grad = (p - Y)[:, np.newaxis] * phi
    grad = (p - Y) * phi  # Equivalent
    return grad


def g_grad_and_hessian(X, Y, h):
    """
    Compute the gradient and the Hessian of the logistic loss
    Does not work for a batch of data because of the outer product
    """
    # For batch data, should work
    # n, d = X.shape
    # phi = np.hstack([np.ones(n, 1), X])
    # dot_product = np.dot(phi, h)
    # p = sigmoid(dot_product)
    # grad = (p - Y) * phi
    # hessian = np.einsum('i,ij,ik->ijk', p * (1 - p), phi, phi)
    # return grad, hessian

    # For a single data point
    phi = np.hstack([np.ones((1,)), X])
    dot_product = np.dot(phi, h)
    p = sigmoid(dot_product)
    grad = (p - Y) * phi
    hessian = p * (1 - p) * np.outer(phi, phi)
    return grad, hessian

In [ ]:
# test broadcast numpy *
a = np.array([[1], [2]])  # like (p - Y) for a batch of 2 samples
X = np.array([[1, 2, 3, 4], [5, 6, 7, 8]])
# print(a * x)

# test outer product
# print(np.outer(x, x))

# test np.atleast_2d
X = np.atleast_2d(np.array([1, 2, 3]))
# print(x.shape)

# test np.dot
X = np.array([[1, 2, 3], [4, 5, 6]])
Y = np.array([[1], [2], [3]])
# print(np.dot(x, y))

# test np.einsum
p = np.array([0.5, 0.6])
X = np.array([[1, 2], [3, 4]])
print(np.einsum("i,ij,ik->ijk", p * (1 - p), X, X))
print(p[0] * (1 - p[0]) * np.outer(X[0], X[0]))
print(p[1] * (1 - p[1]) * np.outer(X[1], X[1]))

In [ ]:
class SGD(Optimizer):
    """
    Stochastic Gradient Descent optimizer
    Uses a learning rate lr = c_mu * iteration^(-mu)
    """

    def __init__(self, mu: float, c_mu: float):
        self.name = "SGD"
        self.mu = mu
        self.c_mu = c_mu
        self.iteration = 0

    def reset(self, theta_dim: int):
        """
        Reset the optimizer state
        """
        self.iteration = 0

    def step(
        self,
        X: np.ndarray,
        Y: np.ndarray,
        theta: np.ndarray,
        g_grad: Callable,
        g_grad_and_hessian: Callable,
    ):
        """
        Perform one optimization step
        """
        self.iteration += 1
        grad = g_grad(X, Y, theta)
        learning_rate = self.c_mu * self.iteration ** (-self.mu)
        theta += -learning_rate * grad

In [ ]:
class SNA(Optimizer):
    """
    Stochastic Newton Algorithm optimizer
    """

    def __init__(self, mu: float, c_mu: float):
        self.name = "SNA"
        self.mu = mu
        self.c_mu = c_mu
        self.iteration = 0

    def reset(self, theta_dim: int):
        """
        Reset the learning rate and estimate of the hessian
        """
        self.iteration = 0
        self.hessian = np.eye(theta_dim)
        self.hessian_inv = np.eye(theta_dim)

    def step(
        self,
        X: np.ndarray,
        Y: np.ndarray,
        theta: np.ndarray,
        g_grad: Callable,
        g_grad_and_hessian: Callable,
    ):
        """
        Perform one optimization step
        """
        self.iteration += 1
        grad, hessian = g_grad_and_hessian(X, Y, theta)
        self.hessian += (hessian - self.hessian) / (self.iteration + 1)
        try:
            self.hessian_inv = np.linalg.inv(self.hessian)
        except np.linalg.LinAlgError:
            print("Hessian is not invertible")
        learning_rate = self.c_mu * self.iteration ** (-self.mu)
        theta += -learning_rate * self.hessian_inv @ grad

In [ ]:
# Usage example
N = 10
n = 10_0000
true_theta = np.array([0, 3, -9, 4, -9, 15, 0, -7, 1, 0])
optimizer_list = [SGD(0.66, 1), SNA(1, 1)]
e = 2

exp = Experiment(
    g,
    g_grad,
    g_grad_and_hessian,
    optimizer_list,
    e,
    true_theta=true_theta,
    generate_dataseet=create_dataset_logistic,
)
exp.generate_initial_theta()
exp.dataset = exp.generate_dataset(n, true_theta)

In [ ]:
# Run the experiment
theta_errors, _ = exp.run(plot=True)

In [ ]:
theta_error_avg, _ = exp.run_multiple(num_runs=N)